# Import Modules

In [ ]:
# General ------------------------------------------------------------------------------------------------------------------
import os
import math
import random
import itertools
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Image Augmentation -------------------------------------------------------------------------------------------------------
import imgaug as ia
import imgaug.augmenters as iaa

# Deep Learning ------------------------------------------------------------------------------------------------------------
import tensorflow as tf
from tensorflow.keras import optimizers, initializers, layers, activations, metrics

# Bayesian Optimization ----------------------------------------------------------------------------------------------------
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

# Settings

In [ ]:
# Set random seeds ---------------------------------------------------------------------------------------------------------
rSeed = 0
random.seed(rSeed)
ia.seed(rSeed)
np.random.seed(rSeed)

# Set the split fraction between train and test sets -----------------------------------------------------------------------
splitFraction = 0.7

# Grid search settings -----------------------------------------------------------------------------------------------------
gridSearchIterationLimit = float("inf")                 # Set to infinity for searching the whole grid
# gridSearchIterationLimit = 10

# Random search settings ---------------------------------------------------------------------------------------------------
randomSearchIterationLimit = 847                        # Set to 847 for a fair comparison between random search and grid search
# randomSearchIterationLimit = 10

# Bayesian optimization settings -------------------------------------------------------------------------------------------
bayeOptIterationLimit = 25
# bayeOptIterationLimit = 10

# Deep learning model settings ---------------------------------------------------------------------------------------------
regEpochs = 5

# Functions

In [ ]:
### Regression Model ---------------------------------------------------------------------------------------------------------------------------------
def createRegModel(lr=-1, beta1=0.9, beta2=0.999, epsilon=None, decay=0.0):
    try: del model
    except:pass
    
    # Create a sequential model --------------------------------------------------------------------------------------------
    model = tf.keras.models.Sequential()
    # Add a convolutional layer --------------------------------------------------------------------------------------------
    model.add(layers.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='valid', input_shape=(41, 41, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='valid'))
    # Add another convolution layer ----------------------------------------------------------------------------------------
    model.add(layers.Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='valid'))
    # Flatten the output ---------------------------------------------------------------------------------------------------
    model.add(layers.Flatten())
    # Add dense layers -----------------------------------------------------------------------------------------------------
#     model.add(layers.Dense(2))
#     model.add(layers.Dense(2))
    model.add(layers.Dense(2))
    # Add a softmex layer --------------------------------------------------------------------------------------------------
    model.add(layers.Softmax())
    # Set ADAM as the training algorithm -----------------------------------------------------------------------------------
    opt = optimizers.Adam(lr=10**lr, beta_1=beta1, beta_2=beta2, epsilon=epsilon, decay=decay)
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=[metrics.CategoricalAccuracy()])
    
    # Print summary of the model -------------------------------------------------------------------------------------------
#     print(model.inputs, model.outputs, model.summary(), sep="\n\n")
    return model
### --------------------------------------------------------------------------------------------------------------------------------------------------



### Image Augmentation -------------------------------------------------------------------------------------------------------------------------------
def augmentImages(origImages):
    seqAug = iaa.Sequential([iaa.OneOf([iaa.AdditiveGaussianNoise(loc=0, scale=(0, 0.1*255)), iaa.GaussianBlur(sigma=(0, 1))]),
                            iaa.Affine(rotate=(-25, 25))])
    augImages = seqAug.augment_images(origImages)
    return augImages
### --------------------------------------------------------------------------------------------------------------------------------------------------



### Bayesian Optimization ----------------------------------------------------------------------------------------------------------------------------
def objectiveFunction(lr=-4, beta1=0.9, beta2=0.999):
    # Create the model using specific hyperparameters
    model = createRegModel(lr=lr, beta1=beta1, beta2=beta2)
    # Train the model
    model.fit_generator(generator=trainBatchIterator, steps_per_epoch=len(trainBatchIterator), epochs=regEpochs)
    # Evaluate model
    modelScore = model.evaluate(trainX, categoricalTrainY)
    # Return accuracy on train set
    return modelScore[1]
### --------------------------------------------------------------------------------------------------------------------------------------------------


### Metrics ------------------------------------------------------------------------------------------------------------------------------------------
def evaluateModel(method, predTestY):
    tp = tf.keras.metrics.TruePositives()
    tn = tf.keras.metrics.TrueNegatives()
    fp = tf.keras.metrics.FalsePositives()
    fn = tf.keras.metrics.FalseNegatives()
    _ = tp.update_state(testY, predTestY)
    _ = tn.update_state(testY, predTestY)   
    _ = fp.update_state(testY, predTestY)
    _ = fn.update_state(testY, predTestY)    
    
    # True Positives -------------------------------------------------------------------------------------------------------
    truePos = tp.result().numpy()
    # True Negatives -------------------------------------------------------------------------------------------------------
    trueNeg = tn.result().numpy()
    # False Positives ------------------------------------------------------------------------------------------------------
    falsePos = fp.result().numpy()
    # False Negatives ------------------------------------------------------------------------------------------------------
    falseNeg = fn.result().numpy()
    
    # Accuracy -------------------------------------------------------------------------------------------------------------
    accuracy = (truePos + trueNeg)/(truePos + trueNeg + falsePos + falseNeg)
    # Sensitivity ----------------------------------------------------------------------------------------------------------
    sensitivity = (truePos)/(truePos + falseNeg)
    # Specificity ----------------------------------------------------------------------------------------------------------
    specificity = (trueNeg)/(trueNeg + falsePos)
    # Precision ------------------------------------------------------------------------------------------------------------
    precision = (truePos)/(truePos + falsePos)
    return method, accuracy, sensitivity, specificity, precision
### --------------------------------------------------------------------------------------------------------------------------------------------------

# Create Train/Test Sets

In [ ]:
# Load dataset -------------------------------------------------------------------------------------------------------------
dataX = scipy.io.loadmat("dataset_41_41_1_13031.mat")["Input"]
dataY = scipy.io.loadmat("dataset_41_41_1_13031.mat")["Target"]

trainX, trainY, testX, testY = [], [], [], []
# Create train and test splits ---------------------------------------------------------------------------------------------
for i in range(dataX.shape[3]):
    if i < math.ceil(dataX.shape[3] * splitFraction):
        trainX.append(dataX[:, :, :, i])
        trainY.append(dataY[:, i])
    else:
        testX.append(dataX[:, :, :, i])
        testY.append(dataY[:, i])
trainX = np.asarray(trainX)
trainY = np.reshape(np.asarray(trainY), (-1,))
testX = np.asarray(testX)
testY = np.reshape(np.asarray(testY), (-1,))

# Augment train set images -------------------------------------------------------------------------------------------------
trainX = np.concatenate((augmentImages(trainX), trainX))
trainY = np.concatenate((trainY, trainY))

# Print shapes of train/test sets ------------------------------------------------------------------------------------------
print("Shape of train data:", trainX.shape)
print("Shape of train labels:", trainY.shape)
print("Shape of test data:", testX.shape)
print("Shape of test label:", testY.shape)


# Convert binary labels into categorical labels ----------------------------------------------------------------------------
categoricalTrainY = np.asarray(tf.keras.utils.to_categorical(trainY, num_classes=2))

# Create image iterator objects --------------------------------------------------------------------------------------------
imgGen = tf.keras.preprocessing.image.ImageDataGenerator()
trainBatchIterator = imgGen.flow(x=trainX, y=categoricalTrainY, batch_size=64, shuffle=True, seed=rSeed)

# Preview Samples

In [ ]:
index = 18
plt.imshow(trainX[index, :, :, 0])
print("This sample belongs to the class:", categoricalTrainY[index])

# Grid Search

In [ ]:
# Hyper-parameter space to probe -------------------------------------------------------------------------------------------
lrList = [-6, -5, -4, -3, -2, -1, 0]
betaList = [0.0001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.9999]

# Implement grid search ----------------------------------------------------------------------------------------------------
bestParameters = 0
maxAccuracy = 0
for i, parameters in enumerate(itertools.product(lrList, betaList, betaList)):
    if i == gridSearchIterationLimit: break
    print("\nIteration:", i+1, "||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    print("{'params': {'beta1': ", parameters[1], ", 'beta2': ", parameters[2], ", 'lr': ", parameters[0], "}}", sep="")
    # Create the model using specific hyperparameters
    model = createRegModel(lr=parameters[0], beta1=parameters[1], beta2=parameters[2])
    # Train the model
    model.fit_generator(generator=trainBatchIterator, steps_per_epoch=len(trainBatchIterator), epochs=regEpochs)
    # Evaluate model
    modelScore = model.evaluate(trainX, categoricalTrainY, verbose=0)
    # Keep a record of parameters that give most accurate performance on train set
    if modelScore[1] > maxAccuracy:
        bestParameters = (i, modelScore[1], parameters)
        maxAccuracy = modelScore[1]

# Store the parameters found using Grid Search -----------------------------------------------------------------------------
gridSearchParameters = bestParameters[2]

In [ ]:
# Print the result ---------------------------------------------------------------------------------------------------------
print("\n")
print("{", "'iteration': ", bestParameters[0]+1, ", 'target': ", bestParameters[1], ", 'params': {'beta1': ", bestParameters[2][1], ", 'beta2': ", bestParameters[2][2], ", 'lr': ", bestParameters[2][0], "}}", sep="")

# Random Search

In [ ]:
# Hyper-parameter space to probe -------------------------------------------------------------------------------------------
parameterSetList = [ (np.random.uniform(low=-6, high=0),
                      np.random.uniform(low=0.0001, high=0.9999),
                      np.random.uniform(low=0.0001, high=0.9999)) for i in range(randomSearchIterationLimit) ]

# Implement random search --------------------------------------------------------------------------------------------------
bestParameters = 0
maxAccuracy = 0
for i, parameters in enumerate(parameterSetList):
    print("\nIteration:", i+1, "||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
    print("{'params': {'beta1': ", parameters[1], ", 'beta2': ", parameters[2], ", 'lr': ", parameters[0], "}}", sep="")
    # Create the model using specific hyperparameters
    model = createRegModel(lr=parameters[0], beta1=parameters[1], beta2=parameters[2])
    # Train the model
    model.fit_generator(generator=trainBatchIterator, steps_per_epoch=len(trainBatchIterator), epochs=regEpochs)
    # Evaluate model
    modelScore = model.evaluate(trainX, categoricalTrainY, verbose=0)
    # Keep a record of parameters that give most accurate performance on train set
    if modelScore[1] > maxAccuracy:
        bestParameters = (i, modelScore[1], parameters)
        maxAccuracy = modelScore[1]

# Store the parameters found using Grid Search -----------------------------------------------------------------------------
randomSearchParameters = bestParameters[2]

In [ ]:
# Print the result ---------------------------------------------------------------------------------------------------------
print("\n")
print("{", "'iteration': ", bestParameters[0]+1, ", 'target': ", bestParameters[1], ", 'params': {'beta1': ", bestParameters[2][1], ", 'beta2': ", bestParameters[2][2], ", 'lr': ", bestParameters[2][0], "}}", sep="")

# Bayesian Optimization

In [ ]:
# Set region to probe ------------------------------------------------------------------------------------------------------
pbounds = {"lr": (-6, 0), "beta1": (0.0001, 0.9999), "beta2": (0.0001, 0.9999)}

# Create the bayesian optimizer --------------------------------------------------------------------------------------------
optimizer = BayesianOptimization(f=objectiveFunction, pbounds=pbounds, random_state=rSeed,)
utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.0)

# Maximize the accuracy ----------------------------------------------------------------------------------------------------
for _ in range(bayeOptIterationLimit):
    nextPoint = optimizer.suggest(utility)
    target = objectiveFunction(**nextPoint)
    optimizer.register(params=nextPoint, target=target)
    print(target, nextPoint, end="\n\n")

# Store the parameters found using Grid Search -----------------------------------------------------------------------------
bayeOptParameters = (optimizer.max["params"]["lr"], optimizer.max["params"]["beta1"], optimizer.max["params"]["beta2"])

In [ ]:
# Print the result ---------------------------------------------------------------------------------------------------------
print("\n")
print(optimizer.max)

# Comparing Results

In [ ]:
# Grid search --------------------------------------------------------------------------------------------------------------
print("Evaluating performance using grid search...", end="")
# Create the model using specific hyperparameters
model = createRegModel(lr=gridSearchParameters[0], beta1=gridSearchParameters[1], beta2=gridSearchParameters[2])
# Train the model
model.fit_generator(generator=trainBatchIterator, steps_per_epoch=len(trainBatchIterator), epochs=regEpochs, verbose=0)
# Evaluate model
gridSearchResults = evaluateModel("Grid Search", np.argmax(model.predict(testX), axis = 1))
print(" done!")


# Random search ------------------------------------------------------------------------------------------------------------
print("Evaluating performance using random search...", end="")
# Create the model using specific hyperparameters
model = createRegModel(lr=randomSearchParameters[0], beta1=randomSearchParameters[1], beta2=randomSearchParameters[2])
# Train the model
model.fit_generator(generator=trainBatchIterator, steps_per_epoch=len(trainBatchIterator), epochs=regEpochs, verbose=0)
# Evaluate model
randomSearchResults = evaluateModel("Random Search", np.argmax(model.predict(testX), axis = 1))
print(" done!")


# Bayesian optimization ----------------------------------------------------------------------------------------------------
print("Evaluating performance using Bayesian optimization...", end="")
# Create the model using specific hyperparameters
model = createRegModel(lr=bayeOptParameters[0], beta1=bayeOptParameters[1], beta2=bayeOptParameters[2])
# Train the model
model.fit_generator(generator=trainBatchIterator, steps_per_epoch=len(trainBatchIterator), epochs=regEpochs, verbose=0)
# Evaluate model
bayeOptResults = evaluateModel("Bayesian Optimization", np.argmax(model.predict(testX), axis = 1))
print(" done!\n")


# Compile results ----------------------------------------------------------------------------------------------------------
resultsDF = pd.DataFrame([gridSearchResults, randomSearchResults, bayeOptResults], columns = ["Method", "Accuracy", "Sensitivity", "Specificity", "Precision"])
print(resultsDF)